```
@作者: 阿凯爱玩机器人
@QQ: 244561792
@微信: xingshunkai
@邮箱: xingshunkai@qq.com
@网址: deepsenserobot.com
@B站: "阿凯爱玩机器人"
```

## 导入依赖

In [1]:
import copy
import numpy as np
import open3d as o3d
# 自定义库
from kyle_robot_toolbox.open3d import *

pybullet build time: Jun  3 2022 02:05:55


## 载入点云

In [2]:
# 载入场景点云
scene_pcd = o3d.io.read_point_cloud(f"./data/example/box_pose/pcd_rect_roi.pcd")

# 载入盒子表面点云
box_panel_pcd = o3d.io.read_point_cloud("./data/example/box_pose/box_panel_pcd.pcd")

# 赋值为灰色
box_panel_pcd.paint_uniform_color([0.5, 0.5, 0.5])

# 点云可视化
o3d.visualization.draw_geometries([scene_pcd], window_name="场景点云")

## 获取点云质心

In [3]:
center_point = box_panel_pcd.get_center()
print(f"盒子上表面质心: {center_point}")

盒子上表面质心: [0.048 0.002 0.224]


In [4]:
# 可视化，在质心位置绘制小球
mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.002)
mesh_sphere.compute_vertex_normals()
# 小球平移
mesh_sphere.translate(center_point.reshape(-1), relative=False)
# 给小球上色为红色
mesh_sphere.paint_uniform_color([0.0, 1.0, 0])
# 创建可视化窗口
draw_geometry([scene_pcd, mesh_sphere],\
                window_name="绘制盒子上表面中心点")

## 创建KDTree

In [5]:
# 创建盒子上表面点云的KDTree
box_panel_kdtree = o3d.geometry.KDTreeFlann(box_panel_pcd)

## 获取体素最近的N个邻居

In [6]:
neighbor_num = 40 # 邻居个数
print(f"寻找距离 中心点{center_point} \n最近的{neighbor_num}个邻居，并将其绘制为蓝色")
k, idx, _ = box_panel_kdtree.search_knn_vector_3d(center_point, neighbor_num)
print(f"实际找到的邻居个数: {k}")

寻找距离 中心点[0.048 0.002 0.224] 
最近的40个邻居，并将其绘制为蓝色
实际找到的邻居个数: 40


In [7]:
print(f"邻居点的索引: {idx}")

邻居点的索引: IntVector[2261, 2260, 2193, 2192, 2329, 2262, 2328, 2194, 2330, 2259, 2125, 2191, 2124, 2327, 2126, 2398, 2263, 2397, 2195, 2399, 2123, 2331, 2258, 2396, 2056, 2190, 2127, 2055, 2326, 2057, 2400, 2467, 2264, 2466, 2122, 2054, 2196, 2468, 2332, 2395]


In [12]:
# 可视化
box_panel_pcd2 = copy.deepcopy(box_panel_pcd)
# 点云上色
np.asarray(box_panel_pcd2.colors)[idx, :] = [0, 0, 1]

draw_geometry([box_panel_pcd2], \
              bk_color=[0.4, 0.8, 0.4], \
              window_name="上表面点云+N个最近邻")

In [13]:
# 根据索引获取选择到的点云
center_neighbor_pcd = box_panel_pcd2.select_by_index(idx)

o3d.visualization.draw_geometries([center_neighbor_pcd],\
                                  window_name="N个最近邻")

## 根据搜索半径进行检索

In [16]:
# 检索半径, 单位m
radius = 0.003
print(f"寻找距离 中心点{center_point} \n半径为{radius}的邻居，并将其绘制为蓝色")
k, idx, _ = box_panel_kdtree.search_radius_vector_3d(center_point, radius)
print(f"实际找到的邻居个数: {k}")

寻找距离 中心点[0.048 0.002 0.224] 
半径为0.003的邻居，并将其绘制为蓝色
实际找到的邻居个数: 149


In [ ]:
# 可视化
box_panel_pcd3 = copy.deepcopy(box_panel_pcd)
# 点云上色
np.asarray(box_panel_pcd3.colors)[idx, :] = [0, 0, 1]

draw_geometry([box_panel_pcd3], \
              bk_color=[0.4, 0.8, 0.4], \
              window_name="上表面点云+根据半径检索的点云")